In [1]:
from jupyter_ipynb.Entropy_and_information.generate_variables import test_granger_with_generated_data,generate_linear_causal_series,generate_bidirectional_causality,generate_common_cause,generate_nonlinear_causality,generate_seasonal_causality

# 1. Linear causal relationship: X -> Y at lags 2 and 4
params = {'n': 200, 'lags': [2, 4], 'coeffs': [0.6, 0.3], 'noise_level': 0.2, 'seed': 42}
df1, results1 = test_granger_with_generated_data(generate_linear_causal_series, params)

# 2. Bidirectional causality: X <-> Y with different strengths
params = {
    'n': 200, 
    'xy_lags': [2], 'xy_coeffs': [0.5],  # X -> Y stronger
    'yx_lags': [3], 'yx_coeffs': [0.2],  # Y -> X weaker
    'seed': 42
}
df2, results2 = test_granger_with_generated_data(generate_bidirectional_causality, params)

# 3. Common cause: Z -> X, Z -> Y
params = {
    'n': 200,
    'xz_lags': [2], 'xz_coeffs': [0.5],
    'yz_lags': [3], 'yz_coeffs': [0.5],
    'seed': 42
}
df3, results3 = test_granger_with_generated_data(generate_common_cause, params)

# 4. Non-linear causality
params = {'n': 200, 'lag': 2, 'noise_level': 0.2, 'seed': 42}
df4, results4 = test_granger_with_generated_data(generate_nonlinear_causality, params)

# 5. Seasonal causality
params = {
    'n': 300, 'period': 24, 'lag': 6, 
    'seasonal_strength': 0.8, 'causal_strength': 0.5, 
    'seed': 42
}
df5, results5 = test_granger_with_generated_data(generate_seasonal_causality, params)

ModuleNotFoundError: No module named 'jupyter_ipynb'